# Aufgabe 5: Separierbarkeit der Gauß-Filterung
Als effizientere Variante kann eine zweidimensionale lineare Filterung auf zwei eindimensionale lineare Filterungen reduziert werden.

Ein linearer zweidimensionaler Filter $A \in \mathbb{R}^{m \times n}$ heißt separierbar, wenn er durch Faltung zweier eindimensionaler Filter dargestellt werden kann:
\begin{align}
 A &= D_1 * D_2, \qquad \text{mit $D_1 \in \mathbb{R}^m, D_2 \in \mathbb{R}^n$}\\
                &= D_1 \cdot D_2^\top,  \qquad \text{(da $D_1,D_2$ Vektoren).}
\end{align}
Somit ergibt sich die Faltung zu
\begin{align}
  I * A &= I * (D_1 * D_2)\\
        &= (I * D_1) * D_2 \qquad \text{(Assoziativität der Faltung)}
\end{align}


Implementieren Sie nun die 2-D-Gaußfilterung als Hintereinanderausführung je eines Gaußfilters in vertikaler und horizontaler Richtung!

Vergleichen Sie die Laufzeiten mit der nicht-separierten Filterung für verschiedene Größen der Filtermaske!

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import urllib.request

%matplotlib inline
import matplotlib.pyplot as plt

import imageio
import numpy as np

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Faltungsmaske
Definieren Sie hier wie in der vorherigen Aufgabe zunächst die Parameter `m` und `sigma` des Filters. Berechnen Sie anschließend eine eindimensionale Filtermaske `A_gauss`!

In [ ]:
m = 7
sigma = m / 5

offset = (m - 1) // 2
# Nur in x-Richtung
A_gauss = np.exp(-(np.arange(-offset, m - offset)**2) / (2 * sigma * sigma))
A_gauss /= np.sum(A_gauss)
# Zu 2-D Array erweitern
A_gauss = A_gauss[None, :]

## 2. Laden des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

Für diese Aufgabe ist es wichtig, das Bild im Fließkommaformat vorliegen zu haben. Konvertieren sie `image` zu einer geeigneten Repräsentation:

In [ ]:
image = np.asarray(image, dtype=np.float32) / 255

## 3. Berechung der Faltung
Setzen Sie hier die Funktion `ex2_convolve` aus der vorherigen Aufgabe ein:

In [ ]:
def ex2_convolve(image, filter_mask):
    convolved_image = np.zeros_like(image)

    offset_y = filter_mask.shape[0] // 2
    offset_x = filter_mask.shape[1] // 2
    
    # Explicitly zero-pad the original image
    image_pad = np.zeros((image.shape[0] + 2 * offset_y, image.shape[1] + 2 * offset_x), dtype=image.dtype)
    image_pad[offset_y:offset_y+image.shape[0], offset_x:offset_x+image.shape[1]] = image
    
    # Convolve
    for cy in range(convolved_image.shape[0]):
        for cx in range(convolved_image.shape[1]):
            # Extract image patch of the same size as the mask centered around the current pixel,
            # multiply it element-wise with the mask, and accumulate the results.
            patch = image_pad[cy:cy+filter_mask.shape[0], cx:cx+filter_mask.shape[1]]
            convolved_image[cy, cx] = np.sum(patch * filter_mask)
    
    
    return convolved_image

## 4. Separierter Gauß-Filter

Berechnen Sie nun das gefaltete Bild durch zwei Aufrufe der obigen Funktion! Tipp: Verwenden Sie die Funktion `transpose` aus dem Paket `numpy`, um die Filtermaske zu transponieren.

In [ ]:
%%time
convolved_image = ex2_convolve(ex2_convolve(image, A_gauss), A_gauss.transpose())

## 5. Darstellung
Um die Wirksamkeit des separierten Gauß-Filters zu überprüfen, stellen Sie `image` und `convolved_image` nebeneinander dar:

In [ ]:
plt.figure('Convolution: image comparison', figsize=(12, 6))
plt.subplot(1,2,1, title='Original Image')
plt.imshow(image, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,2,2, title='Convolved Image')
plt.imshow(convolved_image, cmap='gray', vmin=0, vmax=1)
plt.show()

## 6. Laufzeitvergleich für verschiedene Filtergrößen

Vergleichen Sie die Laufzeit der separierten Filterung mit der der 2D-Filterung für verschieden große Filtermasken!

Zur Zeitmessung können Sie die magische Jupyter-Funktion `%time` (pro Zeile) oder `%%time` (pro Zelle) verwenden.

In [ ]:
sizes = np.arange(3, 100, 2)
times_2d = []
times_sep = []

for m in sizes:
    sigma = m / 5
    offset = (m - 1) // 2
    A_gauss = np.exp(-(np.arange(-offset, m - offset)**2) / (2 * sigma * sigma))
    A_gauss = A_gauss[None, :]
    A_gauss_2d = np.dot(A_gauss.T, A_gauss)
    A_gauss_2d /= A_gauss_2d.sum()
    A_gauss /= A_gauss.sum()
    times_2d.append(min(timeit.repeat(
        'convolved_image = ex2_convolve(image, A_gauss_2d)',
        repeat=5, number=1, globals=globals())))
    times_sep.append(min(timeit.repeat(
        'convolved_image = ex2_convolve(ex2_convolve(image, A_gauss), A_gauss.T)',
        repeat=5, number=1, globals=globals())))

plt.figure(figsize=(10, 6))
plt.title('Runtime Comparison')
plt.plot(sizes, times_2d, label='2-D Convolution')
plt.plot(sizes, times_sep, label='Separated Convolution')
plt.xlabel('Filter Size')
plt.ylabel('Time in Seconds')
plt.grid()
plt.legend()
plt.show()